Stack exchange
Pros
the problems are highly realistic/human like
have multiple references + a builtin rating/quality system because of upvotes
tagged with useful information
 
Cons
it’s easy to lookup
rate limits 10k per day
 
Hugging face
Pros
massive
multilingual
real code

Cons
quality depends on our corruption procedure
maybe lookup
some garbage in the dataset
kinda contrived (it’s more like a fill in the blanks dataset rather than true generation or debugging)
 
DIY
Pros
way more control over the type of problem (eg python, data analysis, pandas)

Cons
probably only good at python and maybe JavaScript
slow
maybe wrong


Testing DIY creation of debug challenges based on LLM

In [1]:
from prompting.llms import vLLMPipeline
model_id = "casperhansen/llama-3-70b-instruct-awq"
device = 'cuda'
gpus = 1
max_allowed_memory_in_gb = 79
pipeline = vLLMPipeline(device=device, model_id=model_id, gpus=gpus, llm_max_allowed_memory_in_gb=max_allowed_memory_in_gb)

/workspace/prompting/.venv/lib/python3.10/site-packages/eth_utils/network.py:44: UserWarning: Network 345 with name 'Yooldo Verse Mainnet' does not have a valid ChainId. eth-typing should be updated with the latest networks.
  warnings.warn(
/workspace/prompting/.venv/lib/python3.10/site-packages/eth_utils/network.py:44: UserWarning: Network 12611 with name 'Astar zkEVM' does not have a valid ChainId. eth-typing should be updated with the latest networks.
  warnings.warn(
/workspace/prompting/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-23 01:53:15,024	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-05-23 01:53:15,099	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip insta

2024-05-23 01:53:15.775 |       INFO      

INFO:bittensor: - Total gpu memory 84.97 GB - 


 |  - Available free memory: 84.53 GB - 


INFO:bittensor: - 93.0% of the GPU memory will be utilized for loading the model to device "cuda". - 


2024-05-23 01:53:15.781 |       INFO      

/workspace/prompting/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


 |  - Total gpu memory 84.97 GB - 
2024-05-23 01:53:15.785 |       INFO       |  - 93.0% of the GPU memory will be utilized for loading the model to device "cuda". - 
WARNING 05-23 01:53:15 config.py:173] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 05-23 01:53:15 llm_engine.py:73] Initializing an LLM engine with config: model='casperhansen/llama-3-70b-instruct-awq', tokenizer='casperhansen/llama-3-70b-instruct-awq', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=awq, seed=0)
INFO 05-23 01:53:15 tokenizer.py:32] For some LLaMA V1 models, initializing the fast tokenizer may take a long time. To reduce the initialization time, consider using 'hf-internal-testing/llama-tokenizer' instead of the original tokenizer.


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 05-23 01:53:45 llm_engine.py:222] # GPU blocks: 5216, # CPU blocks: 819


In [2]:
# Create a vLLM_LLM instance to try and create challenges for coding
from prompting.llms import vLLM_LLM
SYSTEM_PROMPT = "You are a coding teacher. You make examples of code with bugs that students can solve. You do not announce what the bugs are."
vllm = vLLM_LLM(llm_pipeline=pipeline, system_prompt=SYSTEM_PROMPT)

In [3]:
vllm.query(message="""
           Add a bug to the following code:
           ```
    def add_numbers(a, b):
        result = a + b
        print("The sum is:", result)
        return result
    ```""")

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.94s/it]
INFO:bittensor: - vLLM_LLM generated the following output:


Here is the modified code with a bug:

```
def add_numbers(a, b):
    result = a - b
    print("The sum is:", result)
    return result
```

Your turn to find and fix the bug! - 


2024-05-23 01:53:49.741 |       INFO       |  - vLLM_LLM generated the following output:


Here is the modified code with a bug:

```
def add_numbers(a, b):
    result = a - b
    print("The sum is:", result)
    return result
```

Your turn to find and fix the bug! - 
2024-05-23 01:53:54.774 |       INFO       |  - vLLM_LLM generated the following output:


{{ 

The bug in this code is that it is subtracting `b` from `a` instead of adding them together. Here is the corrected code:

```
def add_numbers(a, b):
    result = a + b
    print("The sum is:", result)
    return result
```

Now, the function correctly adds `a` and `b` together and returns the result. }} - 


'\n\nHere is the modified code with a bug:\n\n```\ndef add_numbers(a, b):\n    result = a - b\n    print("The sum is:", result)\n    return result\n```\n\nYour turn to find and fix the bug!'

In [4]:
vllm.query(message="""fix this code def add_numbers(a, b):
    result = a - b
    print("The sum is:", result)
    return result""")

Processed prompts: 100%|██████████| 1/1 [00:05<00:00,  5.01s/it]
INFO:bittensor: - vLLM_LLM generated the following output:


{{ 

The bug in this code is that it is subtracting `b` from `a` instead of adding them together. Here is the corrected code:

```
def add_numbers(a, b):
    result = a + b
    print("The sum is:", result)
    return result
```

Now, the function correctly adds `a` and `b` together and returns the result. }} - 


'\n\n{{ \n\nThe bug in this code is that it is subtracting `b` from `a` instead of adding them together. Here is the corrected code:\n\n```\ndef add_numbers(a, b):\n    result = a + b\n    print("The sum is:", result)\n    return result\n```\n\nNow, the function correctly adds `a` and `b` together and returns the result. }}'

In [5]:
# Successfully Created a code based on verbal queues "vllm.query(message="Create a funciton that calculates the sum of two numbers, but introduce a bug")
# vllm.query(message="""
    #       Add a bug to the following code:
    #        ```
    # def add_numbers(a, b):
    #     result = a + b
    #     print("The sum is:", result)
    #     return result
    # ```""") was also a success, do this more rigorously.

In [8]:
from prompting.tools.datasets.code import  HFCodingDataset
data = HFCodingDataset(seed=42)
data.random()['content']

'/*\n * Copyright (c) 1999, Oracle and/or its affiliates. All rights reserved.\n * DO NOT ALTER OR REMOVE COPYRIGHT NOTICES OR THIS FILE HEADER.\n *\n * This code is free software; you can redistribute it and/or modify it\n * under the terms of the GNU General Public License version 2 only, as\n * published by the Free Software Foundation.  Oracle designates this\n * particular file as subject to the "Classpath" exception as provided\n * by Oracle in the LICENSE file that accompanied this code.\n *\n * This code is distributed in the hope that it will be useful, but WITHOUT\n * ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or\n * FITNESS FOR A PARTICULAR PURPOSE.  See the GNU General Public License\n * version 2 for more details (a copy is included in the LICENSE file that\n * accompanied this code).\n *\n * You should have received a copy of the GNU General Public License version\n * 2 along with this work; if not, write to the Free Software Foundation,\n * Inc., 

In [7]:
data

In [1]:
import bittensor as bt


/workspace/prompting/.venv/lib/python3.10/site-packages/eth_utils/network.py:44: UserWarning: Network 345 with name 'Yooldo Verse Mainnet' does not have a valid ChainId. eth-typing should be updated with the latest networks.
  warnings.warn(
/workspace/prompting/.venv/lib/python3.10/site-packages/eth_utils/network.py:44: UserWarning: Network 12611 with name 'Astar zkEVM' does not have a valid ChainId. eth-typing should be updated with the latest networks.
  warnings.warn(


In [4]:
meta = bt.metagraph(1)


In [13]:
for i in range(100):
    print(meta.last_update[i])

tensor(2543521)
tensor(2541467)
tensor(3013814)
tensor(3199766)
tensor(2537782)
tensor(3199739)
tensor(3199759)
tensor(2543778)
tensor(2541453)
tensor(2543703)
tensor(2541610)
tensor(2542185)
tensor(3199790)
tensor(2523560)
tensor(2543260)
tensor(2543744)
tensor(2542167)
tensor(2542432)
tensor(2543193)
tensor(2542761)
tensor(2543674)
tensor(2543743)
tensor(2542021)
tensor(2543612)
tensor(2533379)
tensor(2541329)
tensor(2539029)
tensor(2543417)
tensor(2543779)
tensor(2536059)
tensor(2543191)
tensor(2524007)
tensor(2537521)
tensor(2543795)
tensor(2543252)
tensor(2461514)
tensor(2541324)
tensor(2541205)
tensor(2537747)
tensor(2542078)
tensor(2543728)
tensor(2543480)
tensor(2543656)
tensor(2542418)
tensor(2402890)
tensor(2540964)
tensor(2543174)
tensor(2543045)
tensor(2543108)
tensor(2542834)
tensor(2865084)
tensor(2542698)
tensor(2542791)
tensor(2497745)
tensor(2543373)
tensor(2542367)
tensor(2543360)
tensor(2539824)
tensor(2543091)
tensor(2543693)
tensor(2441671)
tensor(3161394)
tensor(2

In [14]:
meta.block

Parameter containing:
tensor(3199800)

In [19]:
# Go through the values of meta.last_update, and get the 10 max values
import numpy as np
max_values = np.partition(meta.last_update, -10)[-900:]
max_values


array([2541234, 2538032, 2541840, 2540085, 2542065, 2542650, 2542774,
       2534172, 2543119, 2543119, 2542781, 2542735, 2542011, 2543237,
       2540374, 2543187, 2540974, 2538529, 2541740, 2538503, 2543208,
       2540600, 2540313, 2543073, 2543280, 2540852, 2539040, 2542569,
       2543022, 2541128, 2542105, 2542877, 2542851, 2541277, 2539707,
       2402995, 2541594, 2511388, 2541450, 2543170, 2543183, 2539434,
       2542615, 2542893, 2543260, 2542734, 2542089, 2542916, 2527229,
       2538399, 2541479, 2541212, 2541587, 2540581, 2540522, 2536836,
       2542982, 2540044, 2541273, 2540350, 2541614, 2542531, 2543094,
       2543319, 2540189, 2542643, 2542135, 2543189, 2461523, 2538593,
       2541257, 2542263, 2539004, 2540121, 2538357, 2537444, 2542356,
       2543392, 2543129, 2402942, 2541090, 2541468, 2542525, 2542116,
       2543202, 2530817, 2542665, 2542507, 2511040, 2542438, 2543106,
       2528938, 2542962, 2540093, 2543327, 2542093, 2542765, 2543261,
       2542764, 2542